In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from collections import OrderedDict
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, pairwise_distances
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from time import time
from sklearn.naive_bayes import GaussianNB

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from tqdm import tqdm

In [2]:
df1 = pd.read_csv('foodreviews.csv')
df = df1.head(100000)
df.head(5)

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [3]:
df['UserId'].value_counts()

AY12DBB0U420B     88
A3OXHLG6DIBRW8    79
A1TMAVN4CEM8U8    66
A281NPSIMI1C2R    64
A1YUL9PCJR3JTY    63
                  ..
A2JW1AGR92K2WG     1
A9DGMYUPQTCWA      1
AJBMABYMHW3XU      1
ACWLYLKR2506N      1
A9Q950IPXJR1D      1
Name: UserId, Length: 70396, dtype: int64

In [4]:
df['count'] = df['Text'].apply(lambda n: len(n.split(' ')))
df[['Text', 'count']]

C:\Users\HP\AppData\Roaming\Python\Python39\site-packages\pandas\core\frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


,Text,count
0,I have bought several of the Vitality canned d...,49
1,Product arrived labeled as Jumbo Salted Peanut...,31
2,This is a confection that has been around a fe...,99
3,If you are looking for the secret ingredient i...,43
4,Great taffy at a great price. There was a wid...,30
...,...,...
99995,I just love it and will buy another box when I...,15
99996,My late father in law used to have a rating sy...,194
99997,This is my favorite brand of Korean ramen. It ...,44
99998,"I do like these noodles although, to say they ...",62


In [7]:
df['count'].sum()

8308326

In [8]:
df['count'].max()

2640

In [9]:
df['count'].min()

6

In [10]:
df['count'].mean()

83.08326

In [11]:
# Remove Stop Words
stop = stopwords.words('english')
df['Text'] = df['Text'].apply(
        lambda x: ' '.join(word for word in x.split() if word not in stop))

C:\Users\HP\AppData\Roaming\Python\Python39\site-packages\pandas\core\frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [12]:
df['count1'] = df['Text'].apply(lambda n: len(n.split(' ')))
df[['Text', 'count1']]

,Text,count1
0,I bought several Vitality canned dog food prod...,27
1,Product arrived labeled Jumbo Salted Peanuts.....,19
2,"This confection around centuries. It light, pi...",52
3,If looking secret ingredient Robitussin I beli...,26
4,Great taffy great price. There wide assortment...,15
...,...,...
99995,I love buy another box I'm done last one!,9
99996,My late father law used rating system meals: p...,118
99997,This favorite brand Korean ramen. It spicy use...,23
99998,"I like noodles although, say spicy somewhat un...",36


In [13]:
df['count1'].sum()

4880671

In [16]:
df['count1'].max()

1794

In [17]:
df['count1'].min()

5

In [18]:
df['count1'].mean()

48.80671

In [19]:
tokenizer = RegexpTokenizer(r'\w+')
df['Text'] = df['Text'].apply(lambda x: ' '.join(word for word in tokenizer.tokenize(x)))

C:\Users\HP\AppData\Roaming\Python\Python39\site-packages\pandas\core\frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [20]:
df['count2'] = df['Text'].apply(lambda n: len(n.split(' ')))
df[['Text', 'count2']]

,Text,count2
0,I bought several Vitality canned dog food prod...,27
1,Product arrived labeled Jumbo Salted Peanuts t...,20
2,This confection around centuries It light pill...,51
3,If looking secret ingredient Robitussin I beli...,26
4,Great taffy great price There wide assortment ...,15
...,...,...
99995,I love buy another box I m done last one,10
99996,My late father law used rating system meals pa...,123
99997,This favorite brand Korean ramen It spicy used...,23
99998,I like noodles although say spicy somewhat und...,36


In [21]:
df['count2'].sum()

5118853

In [23]:
df['count2'].max()

1913

In [24]:
df['count2'].min()

5

In [25]:
df['count2'].mean()

51.18853

In [26]:
df['Text'] = df['Text'].apply(lambda x: x.lower())

C:\Users\HP\AppData\Roaming\Python\Python39\site-packages\pandas\core\frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [27]:
df['count3'] = df['Text'].apply(lambda n: len(n.split(' ')))
df[['Text', 'count3']]

,Text,count3
0,i bought several vitality canned dog food prod...,27
1,product arrived labeled jumbo salted peanuts t...,20
2,this confection around centuries it light pill...,51
3,if looking secret ingredient robitussin i beli...,26
4,great taffy great price there wide assortment ...,15
...,...,...
99995,i love buy another box i m done last one,10
99996,my late father law used rating system meals pa...,123
99997,this favorite brand korean ramen it spicy used...,23
99998,i like noodles although say spicy somewhat und...,36


In [33]:
df['Text'][0]

'i bought several vitality canned dog food products found good quality the product looks like stew processed meat smells better my labrador finicky appreciates product better most'

In [34]:
uniqueWords = list(set(" ".join(df['Text']).lower().split(" ")))
count = len(uniqueWords)

In [35]:
count

56095

In [36]:
uniqueWords

['lump',
 'finesse',
 'ashbys',
 'skype',
 'adem',
 'paletable',
 'moutful',
 'robertsons',
 'unneccesary',
 'cowardly',
 'streams',
 'pepperminty',
 'mailing',
 'cataracts',
 'zsweet',
 'microphones',
 'unexplained',
 'antiseizure',
 'triazines',
 'proud',
 'clover',
 'cubes',
 'b000v0yl4u',
 'cross',
 'escargot',
 'throaty',
 'intentional',
 'oloves',
 'glutten',
 'thinkorganic',
 'drummer',
 'mexicans',
 'additive',
 'sarsasparilla',
 'longum',
 'spinachy',
 'availible',
 'excels',
 'hearbeat',
 'dishing',
 'listens',
 'hostas',
 'simon',
 'headachy',
 'manna',
 'proustian',
 'vocally',
 'stayathomemom',
 'moravian',
 'castillo',
 'lulled',
 'supporting',
 '1898',
 'kauai',
 'dislikes',
 'percolators',
 'labrabull',
 'chaetognaths',
 'vocal',
 'availing',
 'fueled',
 'bein',
 'culinary',
 'resellers',
 'celery',
 'daybreak',
 'intesely',
 'wasted',
 'b000ffry3g',
 'becomeing',
 'vox',
 '28lbs',
 'curly',
 'loes',
 'petsmart',
 'merry',
 'pap',
 'b000q37vw6',
 'permeating',
 'purchas